In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../data/'
folder = 'sp_matrix_tag'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site

In [2]:
df = pd.read_pickle(os.path.join(dir_, file_name))
train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name))
train_ = train[['uid', 'tid', 'rating', 'count']]

In [3]:
train_x = sp.load_npz(os.path.join(dir_, folder, 'train_x_' + file_name + '.npz'))
train_y = np.loadtxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')
test_x_sp = sp.load_npz(os.path.join(dir_, folder, 'train_x_' + file_name + '.npz'))
test_y_sp = np.loadtxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')

In [4]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 19
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 218
Tag start position : 39355
Length of vector : 39374
----------------------------------------
Number of unique users : 218
Number of unique tracks : 39137


In [5]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [6]:
x_test = []
for i in tqdm(range(unique_users)):
    user = train_[train_['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)
    

In [7]:
i = 0
sp_rows = []
sp_cols = []
for row in tqdm(x_test):
    sp_rows.append(i)
    sp_cols.append(row[0])
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    
    t = df[df['tid']==row[1]].reset_index()
    tags = t['tags'].iloc[0]
    for tag in tags:
        sp_rows.append(i)
        sp_cols.append(tag_start + tag)
        
    i += 1

KeyboardInterrupt: 

In [3]:
w = np.ones(len(sp_rows))
test_x = sp.csr_matrix((w, (sp_rows, sp_cols)), shape=(len(x_test), l), dtype=float)

NameError: name 'np' is not defined

In [8]:
test_x

<8169189x39374 sparse matrix of type '<class 'numpy.float64'>'
	with 31031788 stored elements in Compressed Sparse Row format>

In [ ]:
sp.save_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name), test_x)

In [8]:
test_x = sp.load_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name + '.npz'))

In [ ]:
order = 3
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.001,
    reg=0.01,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)
predictions = model.predict(test_x)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


 10%|█         | 1/10 [00:01<00:09,  1.06s/epoch]

In [59]:
len(predictions), predictions

(8169189,
 array([1.7340777, 1.8649099, 2.4972453, ..., 1.1457595, 1.2446014,
        1.1243104], dtype=float32))

In [60]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', predictions, False) 
df2[:10]

,uid,tid,rating
0,0,10,1.734078
1,0,15,1.864910
2,0,18,2.497245
3,0,38,1.706746
4,0,41,1.641585
5,0,42,1.456616
6,0,62,2.516741
7,0,67,2.061766
8,0,71,2.080520
9,0,76,2.644073


In [61]:
df2.to_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))

In [62]:
predictions = model.predict(test_x_sp)

In [63]:
print('RMSE: {}'.format(math.sqrt(mean_squared_error(test_y_sp, predictions))))
print('MAE: {}'.format(mean_absolute_error(test_y_sp, predictions)))

RMSE: 1.708833713405184
MAE: 1.4649181588326035


In [64]:
predictions = df2

In [65]:
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:10]

,uid,tid,count,tags,rating
20,0,10,1,"[1, 12]",0.25
25,0,15,12,[0],4.75
29,0,18,7,"[9, 1]",4.25
54,0,38,4,[1],3.00
58,0,41,4,[1],3.00
59,0,42,2,[1],1.50
85,0,62,10,"[0, 12]",4.50
91,0,67,1,"[8, 0]",0.25
96,0,71,2,"[4, 1]",1.50
104,0,76,1,"[7, 12, 2, 5]",0.25


In [66]:
num_user = len(predictions['uid'].unique())
num_user

218

In [67]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.009862385321100921
